In [1]:
import pandas as pd
df = pd.read_csv('AWS-SaaS-Sales.csv')

In [2]:
df.head()

,Row ID,Order ID,Order Date,Date Key,Contact Name,Country,City,Region,Subregion,Customer,Customer ID,Industry,Segment,Product,License,Sales,Quantity,Discount,Profit
0,1,EMEA-2022-152156,11/9/2022,20221109,Nathan Bell,Ireland,Dublin,EMEA,UKIR,Chevron,1017,Energy,SMB,Marketing Suite,16GRM07R1K,261.9600,2,0.00,41.9136
1,2,EMEA-2022-152156,11/9/2022,20221109,Nathan Bell,Ireland,Dublin,EMEA,UKIR,Chevron,1017,Energy,SMB,FinanceHub,QLIW57KZUV,731.9400,3,0.00,219.5820
2,3,AMER-2022-138688,6/13/2022,20220613,Deirdre Bailey,United States,New York City,AMER,NAMER,Phillips 66,1056,Energy,Strategic,FinanceHub,JI6BVL70HQ,14.6200,2,0.00,6.8714
3,4,EMEA-2021-108966,10/11/2021,20211011,Zoe Hodges,Germany,Stuttgart,EMEA,EU-WEST,Royal Dutch Shell,1031,Energy,SMB,ContactMatcher,DE9GJKGD44,957.5775,5,0.45,-383.0310
4,5,EMEA-2021-108966,10/11/2021,20211011,Zoe Hodges,Germany,Stuttgart,EMEA,EU-WEST,Royal Dutch Shell,1031,Energy,SMB,Marketing Suite - Gold,OIF7NY23WD,22.3680,2,0.20,2.5164


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Row ID        9994 non-null   int64  
 1   Order ID      9994 non-null   object 
 2   Order Date    9994 non-null   object 
 3   Date Key      9994 non-null   int64  
 4   Contact Name  9994 non-null   object 
 5   Country       9994 non-null   object 
 6   City          9994 non-null   object 
 7   Region        9994 non-null   object 
 8   Subregion     9994 non-null   object 
 9   Customer      9994 non-null   object 
 10  Customer ID   9994 non-null   int64  
 11  Industry      9994 non-null   object 
 12  Segment       9994 non-null   object 
 13  Product       9994 non-null   object 
 14  License       9994 non-null   object 
 15  Sales         9994 non-null   float64
 16  Quantity      9994 non-null   int64  
 17  Discount      9994 non-null   float64
 18  Profit        9994 non-null 

In [4]:
df.isnull().sum()

Row ID          0
Order ID        0
Order Date      0
Date Key        0
Contact Name    0
Country         0
City            0
Region          0
Subregion       0
Customer        0
Customer ID     0
Industry        0
Segment         0
Product         0
License         0
Sales           0
Quantity        0
Discount        0
Profit          0
dtype: int64

### Clean the Data

* Handle missing values (e.g., fill or drop rows with missing Sales, Customer ID, or Order Date).

In [5]:
df = df.dropna(subset=['Sales', 'Customer ID', 'Order Date'])

* Convert Order Date to datetime format

In [6]:
df['Order Date'] = pd.to_datetime(df['Order Date'])

* Remove duplicates based on Order ID if any.

In [7]:
df = df.drop_duplicates('Order ID')

 **Add Month-Year Column**: For monthly aggregations

In [8]:
df['Month-Year'] = df['Order Date'].dt.to_period('M')

**Calculate Expenses:** Derive expenses using Expenses = Sales - Profit

In [9]:
df['Expenses'] = df['Sales'] - df['Profit']

In [10]:
df.to_csv('cleaned_aws_saas_sales.csv', index=False)

### Calculate Monthly Revenue and Burn Rate
Compute monthly revenue (total sales) and burn rate (total expenses) to understand financial performance.

**1. Monthly Revenue**

In [11]:
monthly_revenue = df.groupby('Month-Year')['Sales'].sum().reset_index()
monthly_revenue.columns = ['Month-Year', 'Revenue']
print(monthly_revenue)

   Month-Year     Revenue
0     2020-01   6126.4360
1     2020-02   2151.4020
2     2020-03  19023.9500
3     2020-04  12415.6560
4     2020-05  15939.3490
5     2020-06  13837.6980
6     2020-07  13562.9440
7     2020-08  14667.0320
8     2020-09  26681.7610
9     2020-10  13501.2590
10    2020-11  33182.7350
11    2020-12  36905.7740
12    2021-01  10876.2680
13    2021-02   7178.1930
14    2021-03  16185.6520
15    2021-04  13873.6365
16    2021-05  14107.2525
17    2021-06  13281.7560
18    2021-07  11818.2870
19    2021-08  18997.2010
20    2021-09  27161.5840
21    2021-10  16833.9855
22    2021-11  39154.5315
23    2021-12  32410.5770
24    2022-01  10871.0010
25    2022-02  17491.5020
26    2022-03  19019.0870
27    2022-04  27335.5680
28    2022-05  35552.8100
29    2022-06  24449.3530
30    2022-07  15027.4020
31    2022-08  15872.5645
32    2022-09  30222.2543
33    2022-10  17129.4360
34    2022-11  41080.0570
35    2022-12  58127.3110
36    2023-01  23298.4960
37    2023-0

Groups data by month (e.g., 2023-01) and sums Sales to get total revenue. reset_index() makes Month-Year a column.

**2. Burn Rate:**

Sum Expenses by Month-Year (burn rate = monthly expenses for a startup with no profit)

In [12]:
monthly_expenses = df.groupby('Month-Year')['Expenses'].sum().reset_index()
monthly_expenses.columns = ['Month-Year', 'Burn Rate']
print(monthly_expenses)

   Month-Year   Burn Rate
0     2020-01   4807.0144
1     2020-02   1600.3778
2     2020-03  17811.5686
3     2020-04  10518.0172
4     2020-05  13823.2739
5     2020-06  11113.2240
6     2020-07  15762.3189
7     2020-08  12278.1754
8     2020-09  24986.7175
9     2020-10  12861.2961
10    2020-11  29923.2461
11    2020-12  31703.8294
12    2021-01  12839.2042
13    2021-02   4956.0734
14    2021-03  13210.8878
15    2021-04  13817.9611
16    2021-05  12245.9928
17    2021-06  11030.2452
18    2021-07  10615.8454
19    2021-08  15148.7450
20    2021-09  25221.9420
21    2021-10  15210.1747
22    2021-11  33302.3950
23    2021-12  29735.0437
24    2022-01   8505.7974
25    2022-02  13412.2642
26    2022-03  19232.3903
27    2022-04  22472.2863
28    2022-05  30506.6724
29    2022-06  21674.2698
30    2022-07  13079.4519
31    2022-08  15478.1191
32    2022-09  27316.4277
33    2022-10  16635.4379
34    2022-11  36294.6261
35    2022-12  45121.1634
36    2023-01  18563.8239
37    2023-0

In [13]:
# Save results to CSV
monthly_revenue.to_csv('monthly_revenue.csv', index=False)
monthly_expenses.to_csv('monthly_burn_rate.csv', index=False)

### Calculate Run Rate

Run rate = Average monthly revenue × 12

In [14]:
avg_monthly_revenue = monthly_revenue['Revenue'].mean()
run_rate = avg_monthly_revenue * 12
print(f"Annual Run Rate: ${run_rate:,.2f}")

Annual Run Rate: $274,965.52


* Explanation: Takes the average of monthly revenue and multiplies by 12 to project yearly revenue. Useful for forecasting.

### Calculate Customer Acquisition Cost (CAC)

The dataset doesn’t provide marketing costs, so assume 30% of Expenses are acquisition-related (adjust if you have real data).

* Calculate **acquisition costs** as 30% of expenses

In [15]:
df['Acquisition Cost'] = df['Expenses'] * 0.3
monthly_acquisition_cost = df.groupby('Month-Year')['Acquisition Cost'].sum().reset_index()

* Find each customer’s first purchase month to count new customers.

In [16]:
first_purchase = df.groupby('Customer ID')['Order Date'].min().reset_index()
first_purchase['Month-Year'] = first_purchase['Order Date'].dt.to_period('M')
new_customers = first_purchase.groupby('Month-Year')['Customer ID'].count().reset_index()
new_customers.columns = ['Month-Year', 'New Customers']

* CAC = Monthly acquisition cost ÷ Number of new customers.

In [17]:
cac = monthly_acquisition_cost.merge(new_customers, on='Month-Year')
cac['CAC'] = cac['Acquisition Cost'] / cac['New Customers']
print(cac[['Month-Year', 'CAC']])

  Month-Year          CAC
0    2020-01    53.411271
1    2020-02    26.672963
2    2020-03   205.518099
3    2020-04   286.855015
4    2020-05   691.163695
5    2020-06   666.793440
6    2020-07  2364.347835
7    2020-08  1227.817540
8    2020-09  7496.015250


In [18]:
cac.to_csv('monthly_CAC.csv', index=False)

### Calculate Lifetime Value (LTV)

Estimate the total revenue a customer generates over their lifespan.

**Assumptions:**

Average revenue per customer is based on total sales per Customer ID.

Customer lifespan is the time between first and last purchase (in months). If data is limited, assume a typical SaaS lifespan (e.g., 24 months).

In [19]:
# Calculate Revenue per Customer:
customer_revenue = df.groupby('Customer ID')['Sales'].sum().reset_index()
avg_revenue_per_customer = customer_revenue['Sales'].mean()

# Estimate Lifespan:
customer_lifespan = df.groupby('Customer ID').agg({
    'Order Date': ['min', 'max']
}).reset_index()
customer_lifespan.columns = ['Customer ID', 'First Purchase', 'Last Purchase']
customer_lifespan['Lifespan'] = (customer_lifespan['Last Purchase'] - customer_lifespan['First Purchase']).dt.days / 30
avg_lifespan = customer_lifespan['Lifespan'].mean()

In [20]:
ltv = avg_revenue_per_customer * avg_lifespan
print(f"Average LTV: ${ltv:,.2f}")

Average LTV: $506,423.59


### Compute LTV:CAC Ratio

Evaluate the efficiency of customer acquisition (healthy ratio > 3).


In [21]:
avg_cac = cac['CAC'].mean()
ltv_cac_ratio = ltv / avg_cac
print(f"LTV:CAC Ratio: {ltv_cac_ratio:.2f}")

LTV:CAC Ratio: 350.10


In [22]:
# Generate a PDF summarizing LTV, CAC, and LTV:CAC ratio

from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

c = canvas.Canvas("LTV_CAC_Report.pdf", pagesize=letter)
c.drawString(100, 750, "Financial KPI Report")
c.drawString(100, 730, f"Average LTV: ${ltv:,.2f}")
c.drawString(100, 710, f"Average CAC: ${avg_cac:,.2f}")
c.drawString(100, 690, f"LTV:CAC Ratio: {ltv_cac_ratio:.2f}")
c.save()

### Perform Cohort Analysis

Track customer behavior (e.g., revenue or retention) by acquisition month.

In [23]:
#Assign Cohorts:
first_purchase = df.groupby('Customer ID')['Order Date'].min().reset_index()
first_purchase['Cohort Month'] = first_purchase['Order Date'].dt.to_period('M')
cohort_data = df.merge(first_purchase[['Customer ID', 'Cohort Month']], on='Customer ID')

# Calculate Cohort Revenue:
cohort_revenue = cohort_data.groupby(['Cohort Month', 'Month-Year'])['Sales'].sum().unstack()

# Calculate Cohort Retention:
cohort_counts = cohort_data.groupby(['Cohort Month', 'Month-Year'])['Customer ID'].nunique().unstack()
cohort_retention = cohort_counts.divide(cohort_counts.iloc[:, 0], axis=0)

In [24]:
cohort_revenue.to_csv('cohort_revenue.csv')
cohort_retention.to_csv('cohort_retention.csv')